%pip install -r requirments.txt --force

In [3]:
from flask_socketio import SocketIO, emit, join_room, leave_room
from flask import *
from flask_login import *
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS

In [5]:
app = Flask(__name__)
app.secret_key = "mkqw2o0@#mk12!mk3"
socketio = SocketIO(app)
lm = LoginManager()
lm.init_app(app)

ngrok = ''

from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

@lm.user_loader
def user_loader(userId):
    userInfo = User.get_user_info(userId)
    return User(userInfo)

@lm.unauthorized_handler
def unauthorized():
    return redirect('/')

#region CLASS

class User(UserMixin):
    def __init__(self, info):
        self.info = info
    
    #region getter

    def get_id(self):
        return self.info['userId']

    #endregion
    
    @staticmethod
    def get_user_info(userId):
        return {'userId': userId}

#endregion

#region route

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chatroom', methods = ['post'])
def chatroom():
    params = request.form
    userId = params['userId']
    roomId = params['roomId']
    
    session['userId'] = userId
    session['roomId'] = roomId
    
    return render_template(
        'chat.html',
        userId = userId,
        roomId = roomId
    )

@app.route('/chatbot')
def chatbot():
    return render_template('RAG.html')

@app.route('/login', methods = ['post'])
def login():
    params = request.get_json()
    userId = params['userId']
    
    userInfo = User.get_user_info(userId)
    login_user(User(userInfo))
    return jsonify({'result': 1})

@app.route('/logout')
def logout():
    logout_user()
    return redirect('/')

@app.route('/pdfprocess', methods=['post'])
def pdfprocess():
    files = [request.files[i] for i in request.files]
    
    text_sum = ''
    
    for file in files:
        reader = PdfReader(file)
        for page in reader.pages: #페이지 별로 텍스트 추출
            text = page.extract_text()
            corrected_text = text.encode('utf-8', errors='ignore').decode('utf-8') #인코딩 오류 무시 및 텍스트 누적
            text_sum += corrected_text +'\n'

    # print(text_sum)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
    splits = text_splitter.split_text(text_sum)
    # print(splits)
    vectorstore = FAISS.from_texts(
        splits,
        embedding = embedding_model
    )
    
    # reader = PdfReader(f)
    # # 총 페이지 수 확인
    # num_pages = len(reader.pages)

    # # 모든 페이지 텍스트 추출
    # all_text = ""
    # for page_num in range(num_pages):
    #     page = reader.pages[page_num]
    #     text = page.extract_text()

    #     # 인코딩 오류 무시 및 텍스트 누적
    #     corrected_text = text.encode('utf-8', errors='ignore').decode('utf-8')
    #     all_text += corrected_text + "\n"  # 페이지 간 텍스트 구분을 위해 줄 바꿈 추가

    # # 추출한 텍스트를 txt 파일로 저장
    # output_file_path = './save.txt'
    # with open(output_file_path, 'w', encoding='utf-8') as output_file:
    #     output_file.write(all_text)

    return jsonify({'result': 1})

#endregion

#region socket

@socketio.on('joined', namespace = '/chatroom')
def chat_joined(d):
    roomId = session.get('roomId')
    print(d, 'joined', roomId)
    join_room(roomId)

@socketio.on('msg', namespace = '/chatroom')
def socket_msg(d):
    roomId = session.get('roomId')
    print('msg', d)
    socketio.emit('msg', d, namespace = '/chatroom', room = roomId)

#endregion

if __name__ == '__main__':
    print('http://192.168.219.51:5100')
    socketio.run(app = app, host = '192.168.219.51', port = 5100, allow_unsafe_werkzeug = True)

http://192.168.219.51:5100


[2024-08-13 14:52:03,617] ERROR in app: Exception on /pdfprocess [POST]
Traceback (most recent call last):
  File "c:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 1826, in full_dispatch_request
    return self.finalize_request(rv)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 1847, in finalize_request
    response = self.process_response(response)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 2344, in process_response
    self.session_interface.save_session(self, ctx.session, response)
  File "c:\Users\SMHRD\anaconda3\Lib\site-packages\flask\sessions.py", line 410, in save_session
    val = self.get_signing_serializer(app).dumps(dict(session))  # type: ignore
      